# Основные операторы PostgreSQL #

Работу выполнил: Самородов Юрий Сергеевич

## Замечания к исходным файлам ##

### Неоптимальный тип данных




Обратил внимание, что значения в поле `list_price` содержат запятые. Из-за этого при импорте данных через скрипт вылетает ошибка:
```
ERROR:  invalid input syntax for type real: "71,49"
CONTEXT:  COPY transaction, line 2, column list_price: "71,49"
```

В задании требуется, чтобы в таблице `transaction` тип полей `list_price` и `standard_price` был `float4 (REAL)`:

<img src="img/float4_req.png" alt="Alt text" width="200"/>


Даннное поле ожидает разделитель "." (точка), однако в исходных таблицах данные в этих полях имеют разделитель "," (запятая)

Через UI DBeaver данные загружаются, ошибки не происходит. Однако запятая оказывается не на своем месте, данные искажаются.

Вот данные из оригинального CSV файла:

<img src="img/wrong_delim_original.png" alt="Alt text" width="500"/>


А вот результат импорта в DBeaver через графический интерфейс:

<img src="img/wrong_delim_placement.png" alt="Alt text" width="350"/>


Разница на лицо.



### Проблема пустых значений

* Строки, где standard_price - пустое значение удалил, так как нулевой себестоимоси быть не может
* Строки, где online_order - пустое значение так же удалил, так как невозможно определить тип транзакции по другим полям
* Разделитель в новом файле transaction_cleared - запятая (отличается от оригинального файла)

### Дата в в виде текста

По условию задачи, поле `DOB` (дата рождения) в таблице `customer` и `transaction_date` в таблице `transaction` должны быть в формате `VARCHAR`:

<img src="img/date_as_string.png" alt="Alt text" width="500"/>

Однако, это не совсем корректно, так как немного усложняет скрипты.
Более правильным подходом было бы хранение даты с форматом DATE , но в рамках данного задания решил конвертировать дату функции `TO_DATE`.

## Предварительная обработка

### Создание таблиц

#### customer

In [ ]:
-- Удаляем таблицу, если существует
DROP TABLE IF EXISTS customer;

--  Создаем таблицу заново
CREATE TABLE customer (
    -- Базовая информация о клиенте
    customer_id INT4 PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    
    -- Персональная информация
    gender VARCHAR(30),
    dob VARCHAR(50),  -- Дата рождения клиента
    
    
    -- Информация о профессии
    job_title VARCHAR(50),
    job_industry_category VARCHAR(50),
    
    -- Финансовое положение клиента
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(50),
    owns_car VARCHAR(30),
    
    -- Место проживания клиента
    address VARCHAR(50),
    postcode VARCHAR(30),
    state VARCHAR(30),
    country VARCHAR(30),
    
    -- Оценка собственности
    property_valuation INT4
);

-- Добавим комментари к таблице
COMMENT ON TABLE customer IS 'Таблица с информацией о клиентах';

-- Add column comments
COMMENT ON COLUMN customer.customer_id IS 'id клиента';
COMMENT ON COLUMN customer.first_name IS 'имя клиента';
COMMENT ON COLUMN customer.last_name IS 'фамилия клиента';
COMMENT ON COLUMN customer.gender IS 'пол';
COMMENT ON COLUMN customer.dob IS 'дата рождения';
COMMENT ON COLUMN customer.job_title IS 'профессия';
COMMENT ON COLUMN customer.job_industry_category IS 'сфера деятельности';
COMMENT ON COLUMN customer.wealth_segment IS 'сегмент благосостояния';
COMMENT ON COLUMN customer.deceased_indicator IS 'флаг актуального клиента';
COMMENT ON COLUMN customer.owns_car IS 'флаг наличия автомобиля';
COMMENT ON COLUMN customer.address IS 'адрес проживания';
COMMENT ON COLUMN customer.postcode IS 'почтовый индекс';
COMMENT ON COLUMN customer.state IS 'штаты';
COMMENT ON COLUMN customer.country IS 'страна проживания';
COMMENT ON COLUMN customer.property_valuation IS 'оценка имущества';

<img src="img/table_create_customer.png" alt="Alt text" width="500"/>


#### transaction

In [ ]:
-- Удаляем таблицу, если существует
DROP TABLE IF EXISTS transaction;

-- Создание таблицы transaction
CREATE TABLE transaction (
    transaction_id INT4,         -- Уникальный идентификатор транзакции
    product_id INT4,            -- Идентификатор продукта
    customer_id INT4,           -- Идентификатор клиента
    transaction_date VARCHAR(30),-- Дата совершения транзакции
    online_order VARCHAR(30),   -- Признак онлайн-заказа
    order_status VARCHAR(30),   -- Статус заказа
    brand VARCHAR(30),          -- Бренд продукта
    product_line VARCHAR(30),   -- Линейка продуктов
    product_class VARCHAR(30),  -- Класс продукта
    product_size VARCHAR(30),   -- Размер продукта
    list_price FLOAT4,         -- Цена по прайс-листу
    standard_cost FLOAT4        -- Стандартная себестоимость продукта
);


-- Добавим комментарий к таблице
COMMENT ON TABLE transaction IS 'Таблица с информацией о транзакциях';

-- Добавим комментарии к столбцам
COMMENT ON COLUMN transaction.transaction_id IS 'Уникальный идентификатор транзакции';
COMMENT ON COLUMN transaction.product_id IS 'Идентификатор продукта';
COMMENT ON COLUMN transaction.customer_id IS 'Идентификатор клиента';
COMMENT ON COLUMN transaction.transaction_date IS 'Дата совершения транзакции';
COMMENT ON COLUMN transaction.online_order IS 'Признак онлайн-заказа';
COMMENT ON COLUMN transaction.order_status IS 'Статус заказа';
COMMENT ON COLUMN transaction.brand IS 'Бренд продукта';
COMMENT ON COLUMN transaction.product_line IS 'Линейка продуктов';
COMMENT ON COLUMN transaction.product_class IS 'Класс продукта';
COMMENT ON COLUMN transaction.product_size IS 'Размер продукта';
COMMENT ON COLUMN transaction.list_price IS 'Цена по прайс-листу';
COMMENT ON COLUMN transaction.standard_cost IS 'Стандартная себестоимость продукта';

<img src="img/table_create_transactions.png" alt="Alt text" width="500"/>

### Очистка таблиц

In [ ]:
-- Удаляем данные
/*
Обратил внимание, что значения в поле list_price содержат запятые. Из-за этого при импорте данных через скрипт вылетает ошибка:
ERROR:  invalid input syntax for type real: "71,49"
CONTEXT:  COPY transaction, line 2, column list_price: "71,49"

При импорте в UI меняется положение запятой, а, следовательно, искажаются данные.
 */

	truncate table customer;
	truncate table transaction;

### Загрузка данных

#### customer

In [ ]:
-- Импортируем данные
	-- в таблицу customer
		COPY customer
		FROM '/Users/yuriy.samorodov/Documents/МФТИ/Семестр 2/Системы Хранения Данных/HW_2/customer.csv' 
		DELIMITER ';' 
		CSV HEADER;



<img src="img/import_customer.png" alt="Alt text" width="500"/>
<br>
<img src="img/import_customer_2.png" alt="Alt text" width="500"/>

#### transaction

In [ ]:
-- в таблицу transaction

/*
float(4) aka real требует точку в качестве разделителя.
Потому нужно либо поменять тип полей list_price и standrard_cost с FLOAT4 (REAL) на NUMERIC или DECIMAL,
но это не удовлетворяет условиям задачи
Либо поменять разделитель прямо в файле, но это неинтересно, так как изменит датасет.
Либо, нужна кастомная функция для преобразования запятых в точки, 
но она требует создания временной таблицы
Поэтому будем импортировать при помощи sed с заменой запятых на точки в режиме реального времени
*/

/*
    * Строки, где standard_price - пустое значение удалил, так как нулевой себестоимоси быть не может
    * Строки, где online_order - пустое значение так же удалил, так как невозможно определить тип транзакции по другим полям
    * Разделитель в новом файле transaction_cleared - запятая (отличается от оригинального файла)
    * 	 
    */

    COPY transaction
    FROM PROGRAM 'sed ''s/,/\./g'' ''/Users/yuriy.samorodov/Documents/МФТИ/Семестр 2/Системы Хранения Данных/HW_2/transaction_csv_clear.csv'''
    WITH (FORMAT csv, DELIMITER ';',  header);


<img src="img/import_transaction_1.png" alt="Alt text" width="500"/>
<br>
<img src="img/import_transaction_2.png" alt="Alt text" width="500"/>


## Задачи и решения

### (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.


In [ ]:
SELECT DISTINCT brand
FROM transaction
WHERE standard_cost > 1500;

<img src="img/task01_dist_brands.png" alt="Alt text" width="200"/>


### (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.


In [ ]:
SELECT *
FROM transaction
WHERE order_status = 'Approved'
    /* данные в исходной таблице хранятся в текстовом виде ДД.ММ.ГГГГ,
     * поэтому дата требует конвертации в правильный формат
     */
    AND TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09';

<img src="img/task02_approved_transactions.png" alt="Alt text" width="500"/>

### (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.


In [ ]:
SELECT DISTINCT job_title
FROM customer
WHERE job_industry_category IN ('IT', 'Financial Services')
    AND job_title LIKE 'Senior%';


<img src="img/task03_seniors.png" alt="Alt text" width="500"/>

### (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services


In [ ]:
SELECT DISTINCT t.brand
FROM transaction t
JOIN customer c ON t.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services';

<img src="img/task04_finance_industry.png" alt="Alt text" width="500"/>

### (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.


In [ ]:
SELECT DISTINCT c.customer_id, c.first_name, c.last_name
FROM transaction t
JOIN customer c ON t.customer_id = c.customer_id
WHERE t.online_order = 'TRUE'
    AND t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
LIMIT 10;

<img src="img/task05_bicycles.png" alt="Alt text" width="500"/>

### (1 балл) Вывести всех клиентов, у которых нет транзакций.


In [ ]:
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
LEFT JOIN transaction t ON c.customer_id = t.customer_id
WHERE t.transaction_id IS NULL;


<img src="img/task06_no_transactions.png" alt="Alt text" width="500"/>

### (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.


In [ ]:
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
JOIN transaction t ON c.customer_id = t.customer_id
WHERE c.job_industry_category = 'IT'
    AND t.standard_cost = (SELECT MAX(standard_cost) FROM transaction);
    

<img src="img/task07_it_max.png" alt="Alt text" width="500"/>

### Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.


In [ ]:
SELECT DISTINCT c.customer_id, c.first_name, c.last_name
FROM customer c
JOIN transaction t ON c.customer_id = t.customer_id
WHERE c.job_industry_category IN ('IT', 'Health')
    AND t.order_status = 'Approved'
    /* данные в исходной таблице хранятся в текстовом виде ДД.ММ.ГГГГ,
     * поэтому дата требует конвертации в правильный формат
     */
    AND TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-07-07' AND '2017-07-17';


<img src="img/task08_it_health_approved.png" alt="Alt text" width="500"/>